## Задание № 6

### Урок 6. Градиентный бустинг (AdaBoost)

#### Задание

1. Для реализованной в методичке модели градиентного бустинга построить графики зависимости ошибки от количества деревьев в ансамбле и от максимальной глубины деревьев. Сделать выводы о зависимости ошибки от этих параметров.
2. **(*)** Модифицировать реализованный алгоритм, чтобы получился стохастический градиентный бустинг. Размер подвыборки принять равным 0.5. Сравнить на одном графике кривые изменения ошибки на тестовой выборке в зависимости от числа итераций.
3. **(*)** Реализовать алгоритм градиетного бустинга, основанный на реализации решающего дерева из ДЗ4 / методички к уроку 4. Сделать выводы о качестве алгоритма по сравнению с реализацией из п.1.

Напишем реализацию градиентного бустинга на основе объектов.

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
import numpy as np

In [49]:
a=[]
def f(b):
    b.append('a')

In [52]:
f(a)
a

['a', 'a', 'a']

Теперь обучим несколько моделей с разными параметрами и исследуем их поведение.

In [40]:
from sklearn.datasets import load_diabetes

In [41]:
X, y = load_diabetes(return_X_y=True)

In [42]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

# Число деревьев в ансамбле
n_trees = 10
# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees
# Максимальная глубина деревьев
max_depth = 3
# Шаг
eta = 1

In [44]:
gb_gboost = GBGradBoost()

trees, train_errors, test_errors = gb_gboost.gb_fit(X_train, X_test, y_train, y_test, coefs, eta)
gb_gboost.evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)


Ошибка алгоритма из 10 деревьев глубиной 3         с шагом 1 на тренировочной выборке: 1005.3393838996096
Ошибка алгоритма из 10 деревьев глубиной 3         с шагом 1 на тестовой выборке: 4908.996560304129


In [20]:
trees[0]

DecisionTreeRegressor(max_depth=3, random_state=42)